In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

NYC Transit Entrance and Exit Data https://data.ny.gov/Transportation/NYC-Transit-Subway-Entrance-And-Exit-Data/i9wp-a4ja/data
https://data.ny.gov/Transportation/NYC-Transit-Subway-Entrance-And-Exit-Data/i9wp-a4ja

In [ ]:
#turnstile = pd.read_csv('./src/NYC_Transit_Subway_Entrance_And_Exit_Data.csv')

In [2]:
def read_mta(file_nums):
    path ='http://web.mta.info/developers/data/nyct/turnstile/turnstile_'
    allFiles = []
    for i in file_nums:
        allFiles.append(path + i + '.txt')
    df_list = [pd.read_csv(file) for file in allFiles]
    df = pd.concat(df_list)
    df.columns = df.columns.str.strip() #Exits Column name has large trailing whitespace
    return df
#get August Data YYMMDD
#Currently mta contains data from September 19, 2020 to April 04, 2020
mta = read_mta(['200919', 
                '200912',
                '200905',
                '200801',
                '200808',
                '200815',
                '200822',
                '200829', 
                '200725',
                '200718',
                '200711',
                '200704',
                '200627',
                '200620',
                '200613',
                '200606',
                '200530',
                '200523',
                '200516',
                '200509',
                '200502',
                '200425',
                '200418',
                '200411',
                '200404',
                ])

In [3]:
mta=mta.reset_index()

In [ ]:
mta.head()

In [ ]:
mta.describe()

In [ ]:
mta.info()

In [ ]:
mta.columns

In [ ]:
mta.head()

In [4]:
#only interested in regular audits. 
mta = pd.DataFrame(mta[mta.DESC == 'REGULAR'])

In [ ]:
mta.head()

In [5]:
mta['datetime'] = pd.to_datetime(mta.DATE+' '+mta.TIME)

In [ ]:
#Needs to group by the individual turnstile... this will give LARGE values when changing turnstiles
#mta['n_in'] = mta.ENTRIES.diff()
#mta['n_out'] = mta.EXITS.diff()

In [ ]:
#n_in and n_out minimum and maximum values are still unreasonable 
#mta.describe()

In [6]:
#to avoid the edge cases as the DF changes turnstiles 
mta['n_in'] = mta.groupby(['STATION', 'UNIT', 'C/A', 'SCP'])['ENTRIES'].diff()
mta['n_out'] = mta.groupby(['STATION', 'UNIT', 'C/A', 'SCP'])['EXITS'].diff()

In [7]:
#filter out negative n_in and n_out
mask = (mta.n_in > 0) & (mta.n_out > 0)

In [8]:
mta = mta[mask]

In [9]:
#create daily entries/exits by Turnstiles
mta['t_daily_entry'] = mta.groupby(['STATION', 'UNIT', 'C/A', 'SCP', 'DATE']).n_in.transform('sum')
mta['t_daily_exit'] = mta.groupby(['STATION', 'UNIT', 'C/A', 'SCP', 'DATE']).n_out.transform('sum')

In [10]:
#create daily entries/exits by Station
mta['s_daily_entry'] = mta.groupby(['STATION', 'DATE']).n_in.transform('sum')
mta['s_daily_exit'] = mta.groupby(['STATION', 'DATE']).n_out.transform('sum')

In [ ]:
mta.n_in.describe()

In [ ]:
mta.sample(5)

In [11]:
mta['t_4hr_traffic'] = mta.n_in + mta.n_out

In [14]:
mta['S_4hr_traffic'] = mta.groupby(['STATION','datetime']).t_4hr_traffic.transform('sum')

In [15]:
mta['dayofweek'] = mta.datetime.apply(lambda x : x.weekday()) 

In [12]:
mta.head()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,datetime,n_in,n_out,t_daily_entry,t_daily_exit,s_daily_entry,s_daily_exit,t_4hr_traffic
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/12/2020,04:00:00,REGULAR,7457067,2535861,2020-09-12 04:00:00,3.0,5.0,291.0,103.0,10709.0,12381.0,8.0
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/12/2020,08:00:00,REGULAR,7457078,2535876,2020-09-12 08:00:00,11.0,15.0,291.0,103.0,10709.0,12381.0,26.0
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/12/2020,12:00:00,REGULAR,7457119,2535920,2020-09-12 12:00:00,41.0,44.0,291.0,103.0,10709.0,12381.0,85.0
5,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/12/2020,16:00:00,REGULAR,7457230,2535953,2020-09-12 16:00:00,111.0,33.0,291.0,103.0,10709.0,12381.0,144.0
6,6,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/12/2020,20:00:00,REGULAR,7457355,2535959,2020-09-12 20:00:00,125.0,6.0,291.0,103.0,10709.0,12381.0,131.0


In [13]:
mta.describe()

,index,ENTRIES,EXITS,n_in,n_out,t_daily_entry,t_daily_exit,s_daily_entry,s_daily_exit,t_4hr_traffic
count,3.586600e+06,3.586600e+06,3.586600e+06,3.586600e+06,3.586600e+06,3.586600e+06,3.586600e+06,3.586600e+06,3.586600e+06,3.586600e+06
mean,1.026908e+05,3.088659e+07,2.262672e+07,1.001133e+04,8.128614e+03,3.629036e+04,2.920792e+04,8.566061e+05,6.806811e+05,1.813995e+04
std,6.039445e+04,1.751202e+08,1.428982e+08,3.403547e+06,3.084302e+06,6.895249e+06,6.287148e+06,3.230612e+07,3.064282e+07,5.992215e+06
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
25%,4.953000e+04,5.798860e+05,3.491368e+05,7.000000e+00,7.000000e+00,6.300000e+01,5.700000e+01,1.183000e+03,1.167000e+03,1.900000e+01
50%,1.008320e+05,2.626718e+06,1.608360e+06,2.000000e+01,2.000000e+01,1.460000e+02,1.320000e+02,2.537000e+03,2.614000e+03,4.800000e+01
75%,1.558410e+05,7.140241e+06,5.058214e+06,5.100000e+01,4.900000e+01,2.980000e+02,2.880000e+02,5.695000e+03,5.924000e+03,1.070000e+02
max,2.176630e+05,2.128764e+09,2.123305e+09,1.895330e+09,1.813831e+09,2.334213e+09,2.348613e+09,2.365671e+09,2.371263e+09,3.242102e+09
